# RLlib Sample Application: MountainCar-v0

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to trains a policy with the `MountainCar-v0` environment:

 - <https://gym.openai.com/envs/MountainCar-v0/>

For more background about this problem, see:

  - ["Efficient memory-based learning for robot control"](https://www.cl.cam.ac.uk/techreports/UCAM-CL-TR-209.pdf)  
[Andrew William Moore](https://www.cl.cam.ac.uk/~awm22/)  
University of Cambridge (1990)
  - ["Solving Mountain Car with Q-Learning"](https://medium.com/@ts1829/solving-mountain-car-with-q-learning-b77bf71b1de2)  
[Tim Sullivan](https://twitter.com/ts_1829)
  
---

First, let's make sure that Ray and RLlib are installed…

In [ ]:
!pip install ray[rllib]
!pip install ray[debug]
!pip install ray[tune]
!pip install pandas
!pip install requests
!pip install tensorflow

Then start Ray…

In [ ]:
import ray
import ray.rllib.agents.ppo as ppo

ray.shutdown()
ray.init(ignore_reinit_error=True)

After a successful launch, the last log output line should read `View the Ray dashboard at localhost:8265`

Open <http://localhost:8265/> in another tab to view the Ray dashboard as the example runs.

---

Next we'll train an RLlib policy with the `MountainCar-v0` environment <https://gym.openai.com/envs/MountainCar-v0/>

By default, training runs for `10` iterations. Increase the `n_iter` setting if you want to see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/moun` directory.

In [ ]:
SELECT_ENV = "MountainCar-v0"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

reward_history = []

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

for _ in range(N_ITER):
    result = agent.train()
    print(result)

    max_reward = result["episode_reward_max"]
    reward_history.append(max_reward)

    file_name = agent.save("/tmp/ppo/moun")
    print(f"\n{file_name}")

In [ ]:
print(reward_history)

Do the episode rewards increase after multiple iterations?
That shows whether the policy is improving.

Also, print out the policy and model to see the results of training in detail…

In [ ]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

This visualizes the "car" agent operating within the simulation: rocking back and forth to gain momentum to overcome the mountain.

In [ ]:
! rllib rollout \
    /tmp/ppo/moun/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"MountainCar-v0\"}" --run PPO \
    --steps 2000

The rollout uses the second saved checkpoint, evaluated through `2000` steps.
Modify the path to view other checkpoints.

---

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) then follow the instructions (copy/paste the URL it generates) to visualize key metrics from training with RLlib…

In [ ]:
!tensorboard --logdir=~/ray_results/